<a href="https://colab.research.google.com/github/taira-no-saurusu/Webdriver/blob/main/ErrorPage_getter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ステータスコードが異常な記事情報（投稿日・記事タイトル・URL・ステータスコード）をcsvで出力・標準出力
**操作方法**

1.   画面上の「ランタイム」タブを選択
2.   「すべてのセルを実行」を選択  
→プログラムが実行され、画面を1番下までスクロールすると、404NotFound等、問題がある記事の情報が順次表示されていく。

<font color="MediumVioletRed">※本プログラムは完了するのに90分近く時間を要するので時間がある時に実行してください</font> 

**主なステータスコード**

*   100~300  >>>  正常(主に200)
*   4xx系    >>>  クライアントエラー
*   403      >>>  閲覧権限なし・アクセス拒否
*   404      >>>  ページが見つからない
*   5xx系    >>>  サーバーエラー  
  
[ステータスコードとは？](https://www.sakurasaku-labo.jp/blogs/status-code-basic-knowledges)









In [ ]:
!apt-get -y install fonts-ipafont-gothic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-ipafont-gothic is already the newest version (00303-18ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
# ライブラリインポート
import os
import shutil
import sys
import time
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import requests
import csv
import datetime

In [ ]:
#開始記事のURL指定(履歴書が不要になる世界へ､｢スクー｣の挑戦)
#この記事をスタートとして順にリンクボタンがあるページを解析していく
URL = 'https://kindaipicks.com/article/000001'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')

driver = webdriver.Chrome('chromedriver',options = options)
driver.get(URL)
time.sleep(1)

In [ ]:
#csvファイル作成
dt_now = datetime.datetime.now()
file_name = str(dt_now.year)+"_"+str(dt_now.month)+"_"+str(dt_now.day)+".csv"
with open(file_name,'w') as csv_file:
    fieldnames = ["投稿日","記事タイトル","URL","ステータスコード"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()


In [ ]:
i = 0;
corrent_url = URL
while True:
    time.sleep(0.5)
    i+=1


    try :
        #HTMLから取得
        readlink = driver.find_element(By.CLASS_NAME,'readLink')

        #class='readLink'からURL取得
        url = readlink.get_attribute('href')

        #レスポンスを取得
        res = requests.get(url)

        #ステータスコードを取得
        status_code = str(res.status_code)

        #print(res.status_code)
        #print(driver.title)
        if status_code != '200' and status_code != "403":

            #記事のタイトルを取得
            title = driver.title

            #記事の投稿日を取得
            posted_date = driver.find_element(By.CLASS_NAME,'date').text
            print("--------------------------------------------------------------------------------")
            print("投稿日："+posted_date)
            print("記事タイトル："+title)
            print("URL:"+corrent_url)
            print("ステータスコード："+status_code)

            #csv書き込み
            with open(file_name,'a') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writerow({'投稿日': posted_date, '記事タイトル':title,'URL':corrent_url,'ステータスコード':status_code})



        #リンクボタンがない記事に関しては次の記事へパスする
    except :
        pass

    try :
        #次の記事のリンクを取得して移動
        prev = driver.find_element(By.CLASS_NAME,'prev')
        next_url_a = prev.find_element(By.TAG_NAME,'a')
        corrent_url=next_url_a.get_attribute('href')
        driver.get(corrent_url)

    #終了条件
    except :
        break

driver.quit()
